In [1]:
import os
import sys
from datetime import datetime
import pandas as pd

from dotenv import load_dotenv

import hopsworks

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:
from src.paths import PARENT_DIR

In [4]:
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_PROJECT_NAME = os.getenv('HOPSWORKS_PROJECT_NAME')
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')

In [5]:
from_year = 2023
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

In [6]:
from src.data import load_raw_data

In [7]:
rides = pd.DataFrame()
for year in range(from_year, to_year + 1):
    rides_one_year = load_raw_data(year)

    # Append the data to the rides DataFrame
    rides = pd.concat([rides, rides_one_year])

2023-01 file is already in local storage
2023-02 file is already in local storage
2023-03 file is already in local storage
2023-04 file is already in local storage
2023-05 file is already in local storage
2023-06 file is already in local storage
2023-07 file is already in local storage
2023-08 file is already in local storage
2023-09 file is already in local storage
2023-10 file is already in local storage
2023-11 file is already in local storage
2023-12 file is already in local storage
Failed to download the file 2024-03
Failed to download the file 2024-04
Failed to download the file 2024-05
Failed to download the file 2024-06
Failed to download the file 2024-07
Failed to download the file 2024-08
Failed to download the file 2024-09
Failed to download the file 2024-10
Failed to download the file 2024-11
Failed to download the file 2024-12


In [8]:
print(rides.shape)

(44281613, 2)


In [9]:
from src.data import transform_raw_data_into_timeseries_data

In [10]:
ts_data = transform_raw_data_into_timeseries_data(rides)

2024-05-08 13:29:54,522 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

2024-05-08 13:29:54,540 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



100%|██████████| 265/265 [00:09<00:00, 27.57it/s]


In [11]:
# Convert the pickup_hour to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# Convert the pickup to a timeseries Unix milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [12]:
ts_data

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2023-01-01 00:00:00+00:00,0,1,1672531200000
1,2023-01-01 01:00:00+00:00,0,1,1672534800000
2,2023-01-01 02:00:00+00:00,0,1,1672538400000
3,2023-01-01 03:00:00+00:00,0,1,1672542000000
4,2023-01-01 04:00:00+00:00,0,1,1672545600000
...,...,...,...,...
2702995,2024-02-29 19:00:00+00:00,1,265,1709233200000
2702996,2024-02-29 20:00:00+00:00,3,265,1709236800000
2702997,2024-02-29 21:00:00+00:00,1,265,1709240400000
2702998,2024-02-29 22:00:00+00:00,2,265,1709244000000


In [13]:
# Login to Hopsworks
project = hopsworks.login(project=HOPSWORKS_PROJECT_NAME, api_key_value=HOPSWORKS_API_KEY)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/699541


In [14]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [15]:
FEATURE_GROUP_NAME = 'timeseries_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [16]:
feature_group = feature_store.get_or_create_feature_group(name=FEATURE_GROUP_NAME, version=FEATURE_GROUP_VERSION,
                                                          description='Timeseries data with hourly frequency', 
                                                          primary_key=['pickup_location_id', 'pickup_ts'], 
                                                          event_time='pickup_hour')

In [17]:
feature_group.insert(ts_data, write_options={'wait_for_job': False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/699541/fs/695364/fg/794975


Uploading Dataframe: 0.00% |          | Rows 0/2703000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: timeseries_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/699541/jobs/named/timeseries_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x132be0790>, None)